In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [30]:
raw_data = pd.read_csv("netflix_titles.csv")


raw_data.info

825

In [31]:
raw_data.isna().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [ ]:
auxiliary_df = raw_data[["director","cast"]]

auxiliary_df["cast_list"] = auxiliary_df["cast"].str.split(",")

auxiliary_df_2 = auxiliary_df.cast_list.apply(pd.Series).fillna("")

auxiliary_df_2["director"] = auxiliary_df["director"]

auxiliary_df_2.dropna()